In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Enter the password here


In [2]:
password = "3me7vdo_"

In [3]:
from npz_loader import load_encrypted_npz

data = load_encrypted_npz(password)

In [4]:
#print("Decrypted data:", data)

# Now start working


In [4]:
X = data['X_train']/255
y = data['y_train']
X_test = data['X_test']/255

In [21]:
np.random.seed(1024)

In [58]:
y.reshape((12700,1))

(12700,)

In [5]:
print(X.shape,'\n',y.shape,'\n',X_test.shape)

(12700, 784) 
 (12700,) 
 (2167, 784)


In [6]:
Df = pd.DataFrame(X)

In [7]:
Df

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
def sigmoid(x):
    return 1./(1.+np.exp(-x))

    
class Binary_Classifier():

    def __init__(self,lr=0.001,n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter
        self.weights = None
        self.bias = None 

    def fit(self,X,y):
        y = y-1
        n_samples , n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        for _ in range(self.n_iter):
            lin_pred = np.dot(X,self.weights) + self.bias
            pred = sigmoid(lin_pred)
    
            dw = (1/n_samples)*np.dot(X.T,(pred - y))
            db = (1/n_samples)*np.sum(pred - y)
    
            self.weights = self.weights - self.lr*dw
            self.bias = self.bias -self.lr*db
    
    def predict(self,X):
        #print(self.weights,self.bias)
        lin_pred = np.dot(X,self.weights) + self.bias
        y_pred = sigmoid(lin_pred)
        #print(y_pred)
#        print(np.bincount(y_pred))
        class_prediction = [1 if y<=0.5 else 2 for y in y_pred]
        return class_prediction

    def split(self,X,y,n_sample=9600):
        n_samples,n_feature = X.shape
        y = y.reshape((n_samples,1))
        concatenated = np.hstack((X,y))
        np.random.shuffle(concatenated)
        X_test,y_test,X_train,y_train = concatenated[:n_sample,:-1],concatenated[:n_sample,-1],concatenated[n_sample:,:-1],concatenated[n_sample:,-1]
        return X_test,y_test,X_train,y_train

    def cross_validation(self,params : list,X_train,y_train,X_test,y_test):
        result = []
        for param in params : 
            self.lr = param[0]
            self.n_iter = param[1]
            self.fit(X_train,y_train)
            y_predicted = self.predict(X_test)
            result.append(accuracy(y_predicted,y_test))
        return result

In [21]:
def accuracy(y_pred,y_test):
    return np.sum(y_pred==y_test)/len(y_test)

In [22]:
log = Binary_Classifier(lr=0.01)

In [23]:
X_train,y_train,X_test2,y_test = log.split(X=X,y=y,n_sample=9000)
print(X_train.shape,y_train.shape,X_test2.shape,y_test.shape)

(9000, 784) (9000,) (3700, 784) (3700,)


In [27]:
params = [(0.01,1000),(0.005,2000),(0.01,2000)]
res = log.cross_validation(params,X_train,y_train,X_test2,y_test)

In [28]:
res

[np.float64(0.98), np.float64(0.98), np.float64(0.9832432432432432)]

In [30]:
maximum = res.index(max(res))
best = params[maximum]
log.lr = best[0]
log.n_iter = best[1]

In [31]:
log.fit(X_train,y_train)

In [32]:
y_predicted = log.predict(X_test2) 
y_predicted = np.array(y_predicted)

In [33]:
y_predicted

array([1, 1, 1, ..., 1, 1, 1])

In [34]:
print(accuracy(y_predicted,y_test))

0.9832432432432432


In [37]:
#This last part is for the unseen data

#log = BinaBinary_Classifier(lr=0.01)
#log.fit(X,y)
log.fit(X,y)
y_definitive = log.predict(X_test)